In [ ]:
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


In [ ]:
path = '../data/nasty_data.fea'
nasty = pd.read_feather(path).sample(frac=.1)

In [ ]:
with open('../models/sz_model.pkl', 'rb') as f:
    sz_model = pickle.load(f)

### Prepare data

In [ ]:
swing_map = {
    'ball': 0,
    'foul': 1,
    'called_strike': 0,
    'swinging_strike': 1,
    'hit_into_play': 1,
    'missed_bunt': 1,
    'blocked_ball': 0,
    'foul_tip': 1,
    'swinging_strike_blocked': 1,
    'foul_bunt': 1,
    'intent_ball': 0,
    'hit_by_pitch': 0,
    'bunt_foul_tip': 1,
    'pitchout': 0,
    'swinging_pitchout': 1}

nasty['swing'] = nasty.description.map(swing_map)

In [ ]:
# nasty['strikezone'] = ((nasty.plate_x <= 17/24) & (nasty.plate_x >= -17/24) &
#                        (nasty.plate_z >= nasty.sz_bot) &
#                        (nasty.plate_z <= nasty.sz_top))\
#     .fillna(False).astype(int)


In [ ]:
# create probabalistic strikezone
sz_model_cols = ['effective_speed', 'pfx_x', 'pfx_z', 'vx0', 'vz0', 'ax',
                 'az', 'release_pos_x', 'release_pos_z']

nasty = nasty.dropna(subset=sz_model_cols).reset_index(drop=True)

X_sz = np.array(nasty[sz_model_cols])
nasty['strikezone_prob'] = sz_model.predict(X_sz)

In [ ]:
# subset to only pitches outside strikezone
# nasty = nasty.loc[nasty.strikezone == 0].reset_index(drop=True)

In [ ]:
# define label and features
label_col = 'swing'
x_cols = ['release_spin_rate', 'effective_speed', 'pfx_x', 'strikezone_prob',
          'pfx_z', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'spin_axis']

nasty = nasty.dropna(subset=[label_col] + x_cols )\
    .reset_index(drop=True)

In [ ]:
y = np.array(nasty[label_col])
X = np.array(nasty[x_cols])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(f'X_train: {len(X_train)}, y_train: {len(y_train)}')
print(f'X_test: {len(X_test)}, y_test: {len(y_test)}')

### Parameter grid

In [ ]:
param_space = {'max_depth': hp.quniform("max_depth", 3, 10, 1),
               'gamma': hp.uniform('gamma', 0, 14),
               'learning_rate': hp.uniform('learning_rate', 0, 1),
               'reg_alpha': hp.uniform('reg_alpha', 0, 1),
               'reg_lambda': hp.quniform('reg_lambda', 0, 10, 1),
               'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1),
               'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
               'n_estimators': hp.quniform('n_estimators', 100, 1000, 10)
               }

evaluation = [(X_train, y_train), (X_test, y_test)]


def objective(space):
    model = xgb.XGBRegressor(objective='binary:logistic',
                             eval_metric='auc',
                             n_estimators=int(space['n_estimators']),
                             learning_rate=space['learning_rate'],
                             max_depth=int(space['max_depth']),
                             gamma=space['gamma'],
                             reg_lambda=space['reg_lambda'],
                             reg_alpha=space['reg_alpha'],
                             min_child_weight=int(space['min_child_weight']),
                             colsample_bytree=space['colsample_bytree']
                             )

    model.fit(X_train, y_train, eval_set=evaluation, eval_metric='auc',
              early_stopping_rounds=10, verbose=False)

    pred = model.predict(X_test, iteration_range=(0, model.best_iteration + 1))
    score = roc_auc_score(y_test, pred)

    return {'loss': -score, 'status': STATUS_OK}


In [ ]:
trials = Trials()
best_hyperparams = fmin(fn=objective,
                        space=param_space,
                        algo=tpe.suggest,
                        max_evals=100,
                        trials=trials)

# # 100%|██████████| 100/100 [55:16<00:00, 33.17s/trial, best loss: -0.7793544321617472]

In [ ]:
model = xgb.XGBRegressor(objective='binary:logistic',
                         use_label_encoder=False,
                         eval_metric='auc',
                         n_estimators=int(best_hyperparams['n_estimators']),
                         learning_rate=best_hyperparams['learning_rate'],
                         max_depth=int(best_hyperparams['max_depth']),
                         gamma=best_hyperparams['gamma'],
                         reg_lambda=best_hyperparams['reg_lambda'],
                         reg_alpha=best_hyperparams['reg_alpha'],
                         min_child_weight=int(best_hyperparams['min_child_weight']),
                         colsample_bytree=best_hyperparams['colsample_bytree'])

model.fit(X_train, y_train, eval_set=evaluation, eval_metric='auc',
          early_stopping_rounds=10, verbose=False)


In [ ]:
xgb.plot_importance(model)